In [1]:
from neo4j import GraphDatabase

In [21]:
from neo4j import GraphDatabase

class Neo4j:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

neo4j = Neo4j("bolt://localhost:7687", "neo4j", "password")

In [22]:
neo4j.driver.execute_query("MATCH (p) RETURN p LIMIT 2")

EagerResult(records=[<Record p=<Node element_id='4:cfc647e9-47d5-478a-941e-2695e1c588f6:0' labels=frozenset({'Program'}) properties={'id': 'AY'}>>, <Record p=<Node element_id='4:cfc647e9-47d5-478a-941e-2695e1c588f6:1' labels=frozenset({'Program'}) properties={'id': 'ILS'}>>], summary=<neo4j._work.summary.ResultSummary object at 0x112d5d6f0>, keys=['p'])

# EF Tokyo Info

Location of IMPORT:
From the Open dropdown menu of your active Neo4j DBMS, select Terminal, and run cd import.

MATCH (n) DETACH DELETE n

LOAD CSV WITH HEADERS FROM 'file:///2_Weekly_Student_Class.csv' AS row
WITH row WHERE row.Prog IS NOT NULL
MERGE (p:Program {id: row.Prog})

LOAD CSV WITH HEADERS FROM 'file:///2_Weekly_Student_Class.csv' AS row
WITH row WHERE row.`Booking No` IS NOT NULL
MERGE (s:Student {id: row.`Booking No`, lastName: row.`Last Name`, firstName: row.`First Name`})

LOAD CSV WITH HEADERS FROM 'file:///2_Weekly_Student_Class.csv' AS row
WITH row WHERE row.Prog IS NOT NULL
MATCH (s:Student {id: row.`Booking No`})
MATCH (p:Program {id:row.Prog})
WITH s,p
MERGE (s)-[r:IN_PROGRAM]->(p)


In [23]:
from datetime import datetime

# Replace with your date
date_string = "2023-12-07"
date_obj = datetime.strptime(date_string, "%Y-%m-%d")

# Getting the ISO week number
iso_calendar = date_obj.isocalendar()
week_number = iso_calendar[1]

print(f"Week number for {date_string} is: {week_number}")


Week number for 2023-12-07 is: 49


In [27]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta, MO

def get_start_end_dates_from_week(year, week_number):
    # Start with the first day of the year
    start_of_year = datetime(year, 1, 1)

    # Find the first Monday of the year
    first_monday = start_of_year + relativedelta(weekday=MO(1))

    # Calculate the start date of the given week
    week_start_date = first_monday + timedelta(weeks=week_number - 1)

    # The end date is 6 days after the start date
    week_end_date = week_start_date + timedelta(days=6)

    return week_start_date, week_end_date




In [41]:
# Example usage
year = 2024
week_number = 2
start_date, end_date = get_start_end_dates_from_week(year, week_number)
print(f"Week {week_number} of {year} starts on {start_date.strftime('%Y-%m-%d')} and ends on {end_date.strftime('%Y-%m-%d')}")


Week 2 of 2024 starts on 2024-01-08 and ends on 2024-01-14


# Google OR-Tools


In [30]:
"""Simple solve."""
from ortools.sat.python import cp_model


def SimpleSatProgram():
    """Minimal CP-SAT example to showcase calling the solver."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, "x")
    y = model.NewIntVar(0, num_vals - 1, "y")
    z = model.NewIntVar(0, num_vals - 1, "z")

    # Creates the constraints.
    model.Add(x != y)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"x = {solver.Value(x)}")
        print(f"y = {solver.Value(y)}")
        print(f"z = {solver.Value(z)}")
    else:
        print("No solution found.")


SimpleSatProgram()

x = 1
y = 0
z = 0


In [33]:
from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f"{v}={self.Value(v)}", end=" ")
        print()

    def solution_count(self):
        return self.__solution_count


def SearchForAllSolutionsSampleSat():
    """Showcases calling the solver to search for all solutions."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, "x")
    y = model.NewIntVar(0, num_vals - 1, "y")
    z = model.NewIntVar(0, num_vals - 1, "z")

    # Create the constraints.
    model.Add(x != y)

    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter([x, y, z])
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True
    # Solve.
    status = solver.Solve(model, solution_printer)

    print(f"Status = {solver.StatusName(status)}")
    print(f"Number of solutions found: {solution_printer.solution_count()}")


SearchForAllSolutionsSampleSat()

x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=2 y=1 z=1 
x=2 y=1 z=0 
x=2 y=1 z=2 
x=2 y=0 z=2 
x=1 y=0 z=2 
x=0 y=1 z=2 
x=0 y=1 z=1 
x=0 y=2 z=1 
x=0 y=2 z=2 
x=1 y=2 z=2 
x=1 y=2 z=1 
x=1 y=2 z=0 
x=0 y=2 z=0 
x=0 y=1 z=0 
Status = OPTIMAL
Number of solutions found: 18


In [34]:
"""Simple solve."""
from ortools.sat.python import cp_model


def main():
    """Minimal CP-SAT example to showcase calling the solver."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    var_upper_bound = max(50, 45, 37)
    x = model.NewIntVar(0, var_upper_bound, "x")
    y = model.NewIntVar(0, var_upper_bound, "y")
    z = model.NewIntVar(0, var_upper_bound, "z")

    # Creates the constraints.
    model.Add(2 * x + 7 * y + 3 * z <= 50)
    model.Add(3 * x - 5 * y + 7 * z <= 45)
    model.Add(5 * x + 2 * y - 6 * z <= 37)

    model.Maximize(2 * x + 2 * y + 3 * z)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"Maximum of objective function: {solver.ObjectiveValue()}\n")
        print(f"x = {solver.Value(x)}")
        print(f"y = {solver.Value(y)}")
        print(f"z = {solver.Value(z)}")
    else:
        print("No solution found.")

    # Statistics.
    print("\nStatistics")
    print(f"  status   : {solver.StatusName(status)}")
    print(f"  conflicts: {solver.NumConflicts()}")
    print(f"  branches : {solver.NumBranches()}")
    print(f"  wall time: {solver.WallTime()} s")


if __name__ == "__main__":
    main()

Maximum of objective function: 35.0

x = 7
y = 3
z = 5

Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 0
  wall time: 0.004396000000000001 s


In [ ]:
# Load SPINs
LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
WITH row WHERE row.FirstName IS NOT NULL
MERGE (s:Student {firstName:row.FirstName, lastName:row.LastName})

LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
WITH row WHERE row.IntensiveChoice03 IS NOT NULL
MERGE (s:SPIN_I {SPIN:row.IntensiveChoice03})

